In [58]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
 
# Instance
driver = webdriver.Chrome()
 

driver.get("https://linkedin.com/uas/login")
time.sleep(2)
 
username = driver.find_element(By.ID, "username")
username.send_keys("EMAIL")  # Enter Your Email Address
 
pword = driver.find_element(By.ID, "password")
pword.send_keys("PASSWORD")        # Enter Your Password
 
driver.find_element(By.XPATH, "//button[@type='submit']").click()

profile_url = "https://www.linkedin.com/company/chelsea-football-club/"

driver.get(profile_url)  

In [59]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

In [26]:
name = soup.find('span', {'dir':'ltr'})
name = name.get_text().strip()
print(name)

Chelsea Football Club


In [60]:
followers = soup.find('div', {'class': 'org-top-card-summary-info-list'})
followers = followers.get_text().strip()
followers = followers.split()
followers = ''.join(filter(str.isdigit, followers[3]))
followers = int(followers.replace(',', ''))
print(followers)

199538


In [28]:
image = soup.find('img', {'class': 'evi-image lazy-image ember-view org-top-card-primary-content__logo'})
image = image.get('src')
print(image)

https://media.licdn.com/dms/image/C4E0BAQGOKE_Fs5llxw/company-logo_200_200/0/1678268962269?e=1704931200&v=beta&t=oIE86jcHXli49FyZIDA5VhGOoGm8IXc5ePHhankaf7U


In [29]:
bio = soup.find('div', {'class':'org-about-module__description'})
bio = bio.get_text().strip()
bio = bio.split()
bio = ' '.join(bio)
print(bio)

Chelsea Football Club, with over a century of rich sporting heritage, has paved the way for a superior collection of hospitality, events and leisure facilities all at one venue. Boasting 25 function rooms and 60 syndicate rooms, alongside t ... see more ... see more


In [72]:
import psycopg2
import uuid
# Replace these values with your PostgreSQL database credentials
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'PASSWORD' #Password of PGadmin
}

# Sample data defined in variables
Account_id = 11
Platform = 'linkedin'
Name = name
Account_url = profile_url
Followers = followers
Profile_image_url = image
Created_date = '2023-10-9'
Updated_date = '2023-10-9'

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # Define the INSERT ... ON CONFLICT ... DO UPDATE query with a unique constraint on account_id
    upsert_query = """
    INSERT INTO social_account_info(Account_id, Platform, Name, Account_url, Followers, Profile_image_url, Bio, Created_date, Updated_date)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (Account_id) DO UPDATE
    SET
        Platform = EXCLUDED.Platform,
        Name = EXCLUDED.Name,
        Account_url = EXCLUDED.Account_url,
        Followers = EXCLUDED.Followers,
        Profile_image_url = EXCLUDED.Profile_image_url,
        Bio = EXCLUDED.Bio,
        Created_date = EXCLUDED.Created_date,
        Updated_date = EXCLUDED.Updated_date;
    """

    # Execute the INSERT ... ON CONFLICT ... DO UPDATE query with the sample data variables
    cursor.execute(upsert_query, (Account_id, Platform, Name, Account_url, Followers, Profile_image_url, Bio, Created_date, Updated_date))

    # Commit the transaction
    conn.commit()
    print(f"Data for Account_id {Account_id} inserted or updated successfully!")

except Exception as e:
    print(f"Error: {str(e)}")

finally:
    # Close the database connection
    cursor.close()
    conn.close()

Data for Account_id 11 inserted or updated successfully!


Data for Account_id c3cf0b9f-2f33-49cb-8566-2da700937709 inserted or updated successfully!
